<a href="https://colab.research.google.com/github/AbdullahFarg/Assignment_1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# "~(for_all(x)[for_all(y)[for_all(z)[imply(imply(P(y),Q(z)),imply(P(x),Q(x)))]]])"


# dic = {
#     "not":
#      {
#          "for_all(x)":
#           {
#               "for_all(y)":
#                {
#                    "for_all(z)":
#                    {
#                        "imply":
#                         (
#                             {"imply": (P(y),Q(z))},
#                             {"imply": (P(x),Q(x))}
#                         )
#                    }
#                }
#           }
#       }
#   }

In [ ]:
def __split (statement) :
    statement.replace( " " , "")
    comp = []
    quantifier, body = statement.split("(", 1)
    body = body[ : -1]
    body += ','
    cur = ""
    c = 0
    for ch in body :
      if(ch == '(') :
        c+=1
      elif(ch == ')') :
        c-=1
      if(ch == ',' and c == 0) :
        comp.append(cur)
        cur = ""
      if(c== 0 and ch in [" " , ","]) :
        continue
      cur+= ch
    return comp

expression = "im(im(P(y) , Q(z)) , im(P(x) , Q(x)))"

print(__split(expression))


['im(P(y) , Q(z))', 'im(P(x) , Q(x))']


In [ ]:
def parse_statement(statement):
    # Base case: If the statement starts with "~", it's a negation
    if statement.startswith("not"):
        quantifier, body = statement.split("(", 1)
        body = body[:-1]
        return {"not": parse_statement(body)}

    # Base case: If the statement starts with a universal quantification (for_all)
    if statement.startswith("for_all"):
        quantifier, body = statement.split("[", 1)
        body = body[:-1]
        return {quantifier: parse_statement(body)}

    # Base case: If the statement starts with an existential quantification (there_exist)
    if statement.startswith("there_exist"):
        quantifier, body = statement.split("[", 1)
        body = body[:-1]
        return {quantifier: parse_statement(body)}

    # Base case: If the statement starts with a conjunction (and)
    if statement.startswith("and"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"and": comps_parsed}

    # Base case: If the statement starts with a disjunction (or)
    if statement.startswith("or"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"or": comps_parsed}

    # Base case: If the statement starts with a implecation (imply)
    if statement.startswith("imply"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"imply": comps_parsed}

    return statement

# Example usage:
statement1 = "not(for_all(x)[for_all(y)[there_exist(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])"
statement1 = "for_all(x)[z]"

# statement = "~(for_all(x)[for_all(y)[there_exist(z)[and(imply(B, and(C, D)), and(E, X), or(A, B))]]])"
parsed_statement = parse_statement(statement1)
print(parsed_statement)


{'for_all(x)': 'z'}


In [ ]:
def Eliminate_implication(my_dict):
    # print(f"my_dict: {my_dict}")
    if not isinstance(my_dict, dict):
        return my_dict

    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    # print(f"first_key: {first_key}")
    # print(f"first_value: {type(first_value)}")
    # print(f"first_value: {first_value}")

    print(f"first_value: {first_value}")
    if isinstance(first_value, list):
      for val in first_value:
          Eliminate_implication(val)
    elif isinstance(first_value, dict):
      for key, val in first_value.items():
          Eliminate_implication(val)

    if "imply" in my_dict:
      my_dict['or'] = my_dict.pop('imply')
      my_dict['or'][0] = {'not': my_dict['or'][0]}

statement2 = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) , Q(z)) , imply(P(x) , Q(x)))]]])"
my_dict1 =  parse_statement(statement2)
# my_dict = {'and': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}
Eliminate_implication(my_dict1)
print(my_dict1)


first_value: {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}
first_value: {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}
first_value: [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]
first_value: ['P(y)', 'Q(z)']
first_value: ['P(x)', 'Q(x)']
{'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'or': [{'not': {'or': [{'not': 'P(y)'}, 'Q(z)']}}, {'or': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}


In [ ]:
# not (a or b) = (not a and not b)
# not (a and b) = (not a or not b)
# not forall() = therE not()
# not thereE() = forall not()

def Move_negation_inward(my_dict) :
    if not isinstance(my_dict, dict) :
          return my_dict

    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    if first_key == "not":
      if isinstance(first_value, str):
        return

      if isinstance(first_value, list):
        for val in first_value:
          val = {"not": val}
          Move_negation_inward(val)
        return

      if isinstance(first_value, dict):
        keys = iter(first_value)
        key = next(keys)
        value = first_value[key]

        if key == "and":
            my_dict["or"] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        elif key == "or":
            my_dict["and"] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        elif key.startswith("for_all"):
            k = "there_exist(" + str(key.split("(", 1)[1][:-1]) + ")"
            my_dict[k] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        elif key.startswith("there_exist"):
            k = "for_all(" + str(key.split("(", 1)[1][:-1]) + ")"
            my_dict[k] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        Move_negation_inward(first_value)
        return
    else:
      if isinstance(first_value, list):
        for val in first_value:
          Move_negation_inward(val)
      elif isinstance(first_value, dict):
        Move_negation_inward(first_value)

# my_dict = {"not": {"for_all(x)": {"there_exist(y)": {"not": {"and": ["A", "B"]}}}}}
my_dict = parse_statement("not(for_all(x)[there_exist(y)[not(and(A, B))]])")
my_dict = parse_statement("not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) , Q(z)) , imply(P(x) , Q(x)))]]])")
print(my_dict)
Eliminate_implication(my_dict)
Move_negation_inward(my_dict)
print(my_dict)
Move_negation_inward(my_dict)
print(my_dict)
Move_negation_inward(my_dict)
print(my_dict)
Move_negation_inward(my_dict)
print(my_dict)


{'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}
first_value: {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}
first_value: {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}
first_value: [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]
first_value: ['P(y)', 'Q(z)']
first_value: ['P(x)', 'Q(x)']
{'for_all(x)': {'there_exist(y)': {'there_exist(z)': {'and': {'not': [{'and': {'not': [{'not': 'P(y)'}, 'Q(z)']}}, {'not': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}
{'for_all(x)': {'there_exist(y)': {'there_exist(z)': {'and': {'not': [{'not': {'not': [{'not': 'P(y)'}, 'Q(z)']}}, {'not': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}
{'for_all(x)': {'there_exist(y)': {'there_exist(z)': {'and': {'not': [{'not': {'not': [{'not': 'P(y)'}, 'Q(z)']}}, {'not': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}
{'for_all(x)': {'there_exist(y)': {

In [ ]:
# Test:1
test1 = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) , Q(z)) , imply(P(x) , Q(x)))]]])"
print(test1)
print("-----------------")

test_my_dict1 =  parse_statement(test1)
print(test_my_dict1)
print("-----------------")

Eliminate_implication(test_my_dict1)
print(test_my_dict1)
print("-----------------")

Move_negation_inward(test_my_dict1)
print(test_my_dict1)
Move_negation_inward(test_my_dict1)
print(test_my_dict1)
Move_negation_inward(test_my_dict1)
print(test_my_dict1)
Move_negation_inward(test_my_dict1)
print(test_my_dict1)

not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) , Q(z)) , imply(P(x) , Q(x)))]]])
-----------------
{'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}
-----------------
first_value: {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}
first_value: {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}
first_value: [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]
first_value: ['P(y)', 'Q(z)']
first_value: ['P(x)', 'Q(x)']
{'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'or': [{'not': {'or': [{'not': 'P(y)'}, 'Q(z)']}}, {'or': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}
-----------------
{'for_all(x)': {'there_exist(y)': {'there_exist(z)': {'and': {'not': [{'and': {'not': [{'not': 'P(y)'}, 'Q(z)']}}, {'not': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}
{'for_all(x)': {'there_exist(y)': {'there_exist(z)':

In [ ]:
def Remove_double_not(my_dict):
    if not isinstance(my_dict, dict):
          return my_dict

    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]
    for val in first_value:
        Remove_double_not(val)



